In [19]:
import pandas as pd
import os, fnmatch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.svm import LinearSVC, SVC
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy.stats
# Random Seed
from numpy.random import seed

test_size_ratio = 0.30
seed(4)

# read the data
df = pd.read_csv("train_test_2019.csv")


In [13]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'y'],
      dtype='object')

In [45]:
X = df[df.columns[:-1]]
y = df.y

cats = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country']
nums = ['age','fnlwgt','capital-gain','capital-loss','hours-per-week']

#X[cats] = pd.Categorical(X[cats])

In [46]:
# one hot encoding
dummies = pd.get_dummies(X[cats])
temp = pd.concat([X,dummies],axis=1)
X = temp.drop(cats,axis=1)
X = X.drop('education-num',axis=1)


In [48]:
for i in nums:
    mean = X[i].mean()
    std = X[i].std()
    X[i]=(X[i]-mean)/std


In [49]:
param_grid ={'C':[0.001,0.01,0.1,1e1,1e2],#C
             'gamma':[0.0001,0.0005,0.001,0.01,0.1,0.5],}#gamma

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_ratio, random_state=None)

svclassifier = GridSearchCV(SVC(kernel='rbf'),param_grid)
svclassifier = svclassifier.fit(X_train,y_train)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [50]:
print("Best: %f using %s" % (svclassifier.best_score_,svclassifier.best_params_))
#    
print("Best estimotor found by grid search:")
y_pred = svclassifier.predict(X_test)
print(predicted_labels)
# Recall - the ability of the classifier to find all the positive samples  TP/(TP+FN)
print("Recall: ", recall_score(y_test, y_pred,average=None))

# Precision - The precision is intuitively the ability of the classifier not to 
#label as positive a sample that is negative          TP/(TP+FP)
print("Precision: ", precision_score(y_test, y_pred,average=None))

# F1-Score - The F1 score can be interpreted as a weighted average of the precision 
#and recall
print("F1-Score: ", f1_score(y_test, y_pred, average=None))

# Accuracy - the number of correctly classified samples
print("Accuracy: %.2f  ," % accuracy_score(y_test, y_pred,normalize=True))
print("Number of samples:",y_test.shape[0])
# Save the model
#joblib.dump(svclassifier, 'svclassifier2.joblib')

Best: 0.846000 using {'C': 10.0, 'gamma': 0.01}
Best estimotor found by grid search:
['no' 'no' 'no' ... 'no' 'yes' 'no']
Recall:  [0.93554163 0.59660574]
Precision:  [0.87119633 0.76039933]
F1-Score:  [0.90222318 0.66861741]
Accuracy: 0.85  ,
Number of samples: 3000
